In [2]:
# prompt: from colab import and mount drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
%cd '/content/drive/MyDrive/Colab Notebooks/oficina_racismo_ambiental/dados/'

/content/drive/MyDrive/Colab Notebooks/oficina_racismo_ambiental/dados


In [14]:
import os
print(os.getcwd())

/content/drive/MyDrive/Colab Notebooks/oficina_racismo_ambiental/dados


In [15]:
!ls


CodigosMuniRJ.csv


In [16]:
# Importando as bibliotecas necessárias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium
import plotly.express as px
import geopandas as gpd
from folium.plugins import HeatMap

In [17]:
# Configurações globais para visualização
pd.options.display.float_format = '{:.2f}'.format
pd.set_option('display.max_columns', None)
plt.style.use("ggplot")

In [18]:
# Configurações globais
crs = {'proj': 'latlong', 'ellps': 'WGS84', 'datum': 'WGS84'}
pd.set_option('display.float_format', '{:.2f}'.format)
plt.style.use("ggplot")

In [19]:
# Funções Utilitárias

def carregar_dados(caminho_populacao, caminho_codigos):
    """Carrega os dados de população e códigos de municípios, realizando o tratamento inicial."""
    # Carregar os dados de população
    populacao = pd.read_excel(caminho_populacao)
    # Carregar os códigos dos municípios
    codigos = pd.read_csv(caminho_codigos).drop(columns=['Unnamed: 0'], errors='ignore')
    # Unir os dados com base no código IBGE
    dados = pd.merge(codigos, populacao, how='left', on='COD_IBGE')
    dados.info()  # Exibir informações básicas dos dados
    return dados

def calcular_populacao_negra(df):
    """Calcula a população negra total e percentual, somando as populações Preta e Parda."""
    df['NEGRA'] = df['PRETA'] + df['PARDA']
    df['NEGRA%'] = (df['NEGRA'] / df['TOTAL']) * 100
    return df

def ajustar_populacao(df, coluna):
    """Ajusta a população total para evitar duplicação de dados."""
    df[coluna] = df[coluna] // 2
    return df

def categorizar_percentual(df, coluna, percentis):
    """Classifica os municípios em faixas de percentuais com base nos percentis fornecidos."""
    categorias = pd.cut(
        df[coluna],
        bins=[0] + list(percentis) + [100],
        labels=[
            'Menor que 35%', '35% a 39%', '40% a 44%', '45% a 49%',
            '50% a 54%', '55% a 59%', '60% a 64%', '65% a 69%',
            '70% a 75%', 'Maior que 75%'
        ]
    )
    df['FAIXA_PERCENTUAL_NEGROS'] = categorias
    return df

def criar_mapa_choropleth(df, coluna_percentual, legenda, localizacao=[-22, -43], zoom_inicial=8):
    """Cria um mapa choropleth para visualização geoespacial dos dados."""
    mapa = folium.Map(location=localizacao, zoom_start=zoom_inicial)
    folium.Choropleth(
        geo_data='https://raw.githubusercontent.com/tbrugz/geodata-br/master/geojson/geojs-33-mun.json',
        name='choropleth',
        data=df,
        columns=['id', coluna_percentual],
        key_on='properties.id',
        fill_color='YlOrBr',
        fill_opacity=0.7,
        line_opacity=0.5,
        legend_name=legenda
    ).add_to(mapa)
    return mapa

def exportar_dados(df, caminho_saida):
    """Exporta os dados para um arquivo Excel, ordenados pelo percentual de população negra."""
    df_ordenado = df.sort_values(by='NEGRA%', ascending=False)
    df_ordenado.to_excel(caminho_saida, index=False)
    print(f"Dados exportados para: {caminho_saida}")

In [20]:
# Definindo os caminhos dos arquivos
caminho_populacao = '/content/drive/MyDrive/Colab Notebooks/oficina_racismo_ambiental/dados/RacismoAmbiental/Pop_Raca_MUNI_RJ_2022.xlsx'
caminho_codigos = '/content/drive/MyDrive/Colab Notebooks/oficina_racismo_ambiental/dados/RacismoAmbiental/CodigosMuniRJ.csv'
caminho_saida = '/content/drive/MyDrive/Colab Notebooks/oficina_racismo_ambiental/dados/RacismoAmbiental/NegrosMunicipios.xlsx'

In [ ]:
# Carregar e tratar os dados
dados = carregar_dados(caminho_populacao, caminho_codigos)
dados = calcular_populacao_negra(dados)
dados = ajustar_populacao(dados, 'POP_TOTAL')

In [ ]:
# Calcular percentis e categorizar
percentis_calculados = np.percentile(dados['NEGRA%'], [33, 35, 40, 45, 50, 55, 60, 65, 70, 75])
dados = categorizar_percentual(dados, 'NEGRA%', percentis_calculados)

In [ ]:
# Exportar dados ordenados por percentual de população negra
exportar_dados(dados, caminho_saida)


In [ ]:
# Visualização da distribuição da população negra com Matplotlib e Seaborn
plt.figure(figsize=(10, 6))
sns.histplot(dados['NEGRA%'], bins=20, kde=True)
plt.title('Distribuição da População Negra (%) por Município')
plt.xlabel('Percentual de População Negra')
plt.ylabel('Frequência')
plt.show()

In [ ]:
# Carregar os dados de esgotamento sanitário
caminho_esgoto = '/content/drive/MyDrive/Colab Notebooks/RacismoAmbiental/Esgotamento_Sanitario_Raca_RJ.xlsx'
df_esgoto = pd.read_excel(caminho_esgoto)
df_esgoto.rename(columns={'CD_MUN': 'id'}, inplace=True)

In [ ]:
# Tratar os dados de esgotamento sanitário
df_esgoto = ajustar_populacao(df_esgoto, 'Pop_Total_Muni')


In [ ]:
# Visualização interativa com Plotly para esgotamento sanitário
fig = px.histogram(df_esgoto, x='Negra%', title='Distribuição de Esgotamento Sanitário por Raça')
fig.show()

In [ ]:
# Criar mapa choropleth para visualização geoespacial
mapa = criar_mapa_choropleth(dados, 'NEGRA%', 'Percentual de População Negra')
mapa.save('/content/drive/MyDrive/Colab Notebooks/RacismoAmbiental/MapaNegrosMunicipiosRJ.html')

In [ ]:
# Funções Utilitárias

def carregar_dados_geograficos(caminho_geojson):
    """Carrega os dados geográficos a partir de um arquivo GeoJSON."""
    gdf = gpd.read_file(caminho_geojson)
    gdf['CD_MUN'] = gdf['CD_MUN'].astype(int)
    print(f"Dados geográficos carregados: {gdf.shape[0]} registros.")
    return gdf

def carregar_dados_estatisticos(caminho_csv, caminho_excel1, caminho_excel2):
    """Carrega os dados estatísticos de esgotamento, lixo e água."""
    esgoto = pd.read_csv(caminho_csv)
    lixo = pd.read_excel(caminho_excel1)
    agua = pd.read_excel(caminho_excel2)
    print("Dados estatísticos carregados.")
    return esgoto, lixo, agua

def merge_dados_geograficos(gdf, df_estatistico, coluna_geografica, coluna_estatistica):
    """Faz o merge dos dados geográficos com os estatísticos."""
    merged = gdf.merge(df_estatistico, left_on=coluna_geografica, right_on=coluna_estatistica, how='left')
    print(f"Merge realizado: {merged.shape[0]} registros.")
    return merged

def tratar_percentuais(df, colunas):
    """Converte as colunas de percentual para o tipo float e substitui vírgulas por pontos."""
    for coluna in colunas:
        df[coluna] = df[coluna].str.replace(',', '.').astype(float)
    return df

def calcular_centroides(df):
    """Calcula os centroides das geometrias e adiciona colunas de latitude e longitude."""
    df['latitude'] = df['geometry'].centroid.y
    df['longitude'] = df['geometry'].centroid.x
    return df